In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import requests
sys.path.append(os.path.abspath('../src'))

from load_data import load_raw
from preprocess import clean_data, select_column, rename, sales_and_residential, drop_na, add_cities, make_zero, geocode, geo_address, one_hot
from train_model import train_random_forest
from evaluate import evaluate_model, compute_shap_local, compute_shap_global


Step 1: Load data and delete the data that is not our target cities

In [ ]:
import pandas as pd

path = r"C:\Users\ktlee\Downloads\all_transactions.csv"
df_raw = load_raw(path)



In [ ]:
df_renamed = rename(df_raw)
df_selected = select_column(df_renamed)
df_filtered = sales_and_residential(df_selected)
df_dropped = drop_na(df_filtered)
df_with_city = add_cities(df_dropped)
df_with_zero = make_zero(df_with_city)



In [ ]:
#Externally download the street data
df_with_zero['Full addr'] = (
        df_with_zero['Street number'].astype(str).str.strip() + ' ' +
        df_with_zero['Street name'].astype(str).str.strip() + ', ' +
        df_with_zero['Postal code'].astype(str).str.strip() + ' ' +
        df_with_zero['Municipality'].astype(str).str.strip()
    )
df = df_with_zero.copy()
df.drop(columns=['Street number', 'Street name', 'Postal code', 'Municipality'], inplace=True)
#df.to_csv(r"C:\Users\ktlee\Downloads\clean_data_new.csv", index = False)

In [ ]:
#Add back the longtitude and latitude
df_address = pd.read_csv(r"C:\Users\ktlee\Downloads\clean_address_dict.csv")
df_address.columns = ['Full addr', 'Longitude', 'Latitude']
#df_address.head
#df_with_zero.columns
df_geocoded = df_with_zero.merge(df_address, on = 'Full addr', how = 'left')

df_clean_geo = df_geocoded[
    (df_geocoded['Longitude'].notna() & df_geocoded['Latitude'].notna()) &
    (df_geocoded['Longitude'] != 0) & (df_geocoded['Latitude'] != 0)
]


In [ ]:
df_clean_geo

In [55]:
#One-hot coding
df_all_eng = one_hot(df_clean_geo)
df_all_eng


,Date of mutation,Property value,Street number,Street name,Postal code,Municipality,Number of rooms,Land area,Type of land use,Actual built surface,...,Latitude,Year,Type of property_Apartment,Type of property_House,Nature of mutation_Sale,Nature of mutation_Sale of future completion,Quarter_1,Quarter_2,Quarter_3,Quarter_4
1,2023-06-12,"135000,00",22.0,DE LA FORGE,6260.0,SAINT-PIERRE,4.0,25.0,S,59.0,...,47.012628,2023,False,True,True,False,False,True,False,False
2,2023-07-07,"55000,00",7022.0,LE VILLAGE,6260.0,SAINT-PIERRE,2.0,64.0,S,41.0,...,47.012628,2023,False,True,True,False,False,False,True,False
4,2023-09-15,"285000,00",5007.0,CHAMP LONG,6260.0,SAINT-PIERRE,6.0,1398.0,T,87.0,...,44.445457,2023,False,True,True,False,False,False,True,False
5,2023-09-15,"285000,00",5007.0,CHAMP LONG,6260.0,SAINT-PIERRE,6.0,1000.0,S,87.0,...,44.445457,2023,False,True,True,False,False,False,True,False
6,2023-01-02,"223000,00",17.0,FRANCOIS DE MONLEON,6190.0,ROQUEBRUNE CAP MARTIN,1.0,0.0,NaN,30.0,...,43.761588,2023,True,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304579,2025-06-23,"835000,00",167.0,SAINT MAUR,75011.0,PARIS 11,4.0,0.0,NaN,70.0,...,48.864419,2025,True,False,True,False,False,True,False,False
304580,2025-06-25,"375000,00",4.0,FABRE D EGLANTINE,75012.0,PARIS 12,1.0,0.0,NaN,33.0,...,48.846491,2025,True,False,True,False,False,True,False,False
304581,2025-06-25,"1370000,00",2.0,PAUL DOUMER,75016.0,PARIS 16,2.0,0.0,NaN,92.0,...,48.859986,2025,True,False,True,False,False,True,False,False
304582,2025-06-27,"550000,00",84.0,VERGNIAUD,75013.0,PARIS 13,3.0,0.0,NaN,61.0,...,48.826399,2025,True,False,True,False,False,True,False,False


Step 2: For each city, pre-process and train

In [ ]:
# After df_with_city (Samuel will add lat/lon later)
df = df_with_city.copy()

# For now, use sample for testing
df_sample = df.head(1000).copy()  # Remove later
df_sample = geocode(df_sample)   # Test geocoding

metrics_table = dict()

training_city = ['Lyon']

for city in cities:
    if city not in training_city:
        continue
    df_city = df[df['City'] == city].copy()
    X, y = clean_data(df_city)

    case_model = 3 #<-- Change this

    if case_model == 3:
        print(X)
        continue

    # Evaluate
    metrics = evaluate_model(model, X_test, y_test)
    metrics_table[city] = metrics
    print(f"City {city}'s model training is done.")

Step 4: SHAP analysis

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X)